In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F 

import pandas as pd

In [3]:
spark = SparkSession.builder.appName("Read CSV with PySpark").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/09/13 09:30:44 WARN Utils: Your hostname, GIGABYTENEJ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/13 09:30:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/13 09:30:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Agg_Departments

In [4]:
# Données
df = pd.DataFrame({
    'patient_id': [1, 2, 3, 4, 5],
    'age': [34, 45, 23, 64, 52],
    'department': ['Cardiology', 'Neurology', 'Cardiology', 'Orthopedics', 'Cardiology'],
    'visit_count': [10, 12, 5, 8, 9]
})

# GroupBy et calculs statistiques
agg_df = df.groupby('department').agg({
    'visit_count': 'sum',
    'age': ['mean', 'max']
}).reset_index()

agg_df

department visit_count        age    
                       sum       mean max
0   Cardiology          24  36.333333  52
1    Neurology          12  45.000000  45
2  Orthopedics           8  64.000000  64

In [22]:
df.dtypes

patient_id       int64
age              int64
department      object
age_category    object
dtype: object

### creation du dataframe en pyspark

In [6]:
emp = [(1, 34, "Cardiology", 10), \
    (2,45,"Neurology",12), \
    (3,23,"Cardiology",5), \
    (4,64,"Orthopedics",8), \
    (5,52,"Cardiology",9)
  ]
empColumns = ["patient_id","age","department","visit_count"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

root
 |-- patient_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- department: string (nullable = true)
 |-- visit_count: long (nullable = true)



+----------+---+-----------+-----------+
|patient_id|age|department |visit_count|
+----------+---+-----------+-----------+
|1         |34 |Cardiology |10         |
|2         |45 |Neurology  |12         |
|3         |23 |Cardiology |5          |
|4         |64 |Orthopedics|8          |
|5         |52 |Cardiology |9          |
+----------+---+-----------+-----------+



### groupby et calcul statistique en pyspark

In [24]:
group = (
    empDF
    .groupBy("department")
    .agg(
        F.sum(F.col('visit_count')),
        F.avg(F.col('age')),
        F.max(F.col('age'))
    )
)
group.show(truncate=False)

+-----------+----------------+------------------+--------+
|department |sum(visit_count)|avg(age)          |max(age)|
+-----------+----------------+------------------+--------+
|Cardiology |24              |36.333333333333336|52      |
|Neurology  |12              |45.0              |45      |
|Orthopedics|8               |64.0              |64      |
+-----------+----------------+------------------+--------+



#### test des aggregations

## Concat Medical Info

In [8]:
# Données
df = pd.DataFrame(
    {
        'patient_name': ['John Doe', 'Jane Smith', 'Alice Brown'],
        'diagnosis': ['Diabetes', 'Heart Disease', 'Hypertension'],
    }
)

# Conversion en minuscules et ajout d'un champ
df['diagnosis_lower'] = df['diagnosis'].str.lower()
df['full_info'] = df['patient_name'] + ' - ' + df['diagnosis_lower']

df


,patient_name,diagnosis,diagnosis_lower,full_info
0,John Doe,Diabetes,diabetes,John Doe - diabetes
1,Jane Smith,Heart Disease,heart disease,Jane Smith - heart disease
2,Alice Brown,Hypertension,hypertension,Alice Brown - hypertension


In [9]:
medInfo = [("John Doe", "Diabetes"), \
    ("Jane Smith","Heart Disease"), \
    ("Alice Brown","Hypertension")
  ]
empColumns = ["patient_name","diagnosis"]

medInfo = spark.createDataFrame(data=medInfo, schema = empColumns)
medInfo.printSchema()
medInfo.show(truncate=False)

root
 |-- patient_name: string (nullable = true)
 |-- diagnosis: string (nullable = true)

+------------+-------------+
|patient_name|diagnosis    |
+------------+-------------+
|John Doe    |Diabetes     |
|Jane Smith  |Heart Disease|
|Alice Brown |Hypertension |
+------------+-------------+



In [10]:
fullDf = medInfo.withColumn(
    'diagnosis_lower',
        F.lower(
            F.col("diagnosis")
            )
        )
    

In [11]:
fullDf2 = fullDf.withColumn(
        'full_info', F.concat(
            F.col(
                "patient_name"
            ),
            F.lit(
                " - "
                ),
            F.col("diagnosis")
        )
    )    

fullDf2.show()

+------------+-------------+---------------+--------------------+
|patient_name|    diagnosis|diagnosis_lower|           full_info|
+------------+-------------+---------------+--------------------+
|    John Doe|     Diabetes|       diabetes| John Doe - Diabetes|
|  Jane Smith|Heart Disease|  heart disease|Jane Smith - Hear...|
| Alice Brown| Hypertension|   hypertension|Alice Brown - Hyp...|
+------------+-------------+---------------+--------------------+



#### test concat_medicalinfo

## Conditionnal calculation

In [12]:
# Données
df = pd.DataFrame(
    {
        'patient_id': [1, 2, 3, 4, 5],
        'age': [34, 70, 50, 20, 15],
        'department': [
            'Cardiology',
            'Neurology',
            'Orthopedics',
            'Cardiology',
            'Neurology',
        ],
    }
)

# Ajout d'une colonne conditionnelle (catégorie d'âge)
df['age_category'] = df['age'].apply(
    lambda x: 'senior' if x > 60 else 'adult' if x > 18 else 'minor'
)

df

,patient_id,age,department,age_category
0,1,34,Cardiology,adult
1,2,70,Neurology,senior
2,3,50,Orthopedics,adult
3,4,20,Cardiology,adult
4,5,15,Neurology,minor


In [13]:
dF = [(1,34,"Cardiology"), \
           (2,70,"Neurology"), \
           (3,50,"Orthopedics"),\
           (4,20,"Cardiology"),\
           (5,15,"Neurology")  
  ]
empColumns = ["patient_id","age","diagnosis"]

dF = spark.createDataFrame(data=dF, schema = empColumns)
dF.printSchema()
dF.show(truncate=False)

root
 |-- patient_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- diagnosis: string (nullable = true)

+----------+---+-----------+
|patient_id|age|diagnosis  |
+----------+---+-----------+
|1         |34 |Cardiology |
|2         |70 |Neurology  |
|3         |50 |Orthopedics|
|4         |20 |Cardiology |
|5         |15 |Neurology  |
+----------+---+-----------+



In [14]:
dF2 = dF.withColumn("age_category", 
                    F.when(
                        dF.age >= 60,"Senior"
                        )
                    .when(
                        dF.age >18 ,"Adult"
                        )
                    .otherwise("Minor")
                    )
dF2.show()

+----------+---+-----------+------------+
|patient_id|age|  diagnosis|age_category|
+----------+---+-----------+------------+
|         1| 34| Cardiology|       Adult|
|         2| 70|  Neurology|      Senior|
|         3| 50|Orthopedics|       Adult|
|         4| 20| Cardiology|       Adult|
|         5| 15|  Neurology|       Minor|
+----------+---+-----------+------------+



#### test conditionnal calculation

## data filteringPatient

In [15]:
filtreDf  = dF.filter(dF.age > 30)
filtreDf.show()

+----------+---+-----------+
|patient_id|age|  diagnosis|
+----------+---+-----------+
|         1| 34| Cardiology|
|         2| 70|  Neurology|
|         3| 50|Orthopedics|
+----------+---+-----------+



## Missing values

In [17]:
ddF = [(1,34,"Cardiology"), \
           (2,None,"Neurology"), \
           (3,50,"Orthopedics"),\
           (4,None,None),\
           (5,15,"Neurology")  
  ]
empColumns = ["patient_id","age","diagnosis"]

ddF = spark.createDataFrame(data=ddF, schema = empColumns)
ddF.printSchema()
ddF.show(truncate=False)

root
 |-- patient_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- diagnosis: string (nullable = true)

+----------+----+-----------+
|patient_id|age |diagnosis  |
+----------+----+-----------+
|1         |34  |Cardiology |
|2         |NULL|Neurology  |
|3         |50  |Orthopedics|
|4         |NULL|NULL       |
|5         |15  |Neurology  |
+----------+----+-----------+



In [18]:
ddF = [(1,34,"Cardiology"), \
           (2,None,"Neurology"), \
           (3,50,"Orthopedics"),\
           (4,None,None),\
           (5,15,"Neurology")  
  ]
empColumns = ["patient_id","age","diagnosis"]

ddF = spark.createDataFrame(data=ddF, schema = empColumns)
ddF.printSchema()
ddF.show(truncate=False)

root
 |-- patient_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- diagnosis: string (nullable = true)

+----------+----+-----------+
|patient_id|age |diagnosis  |
+----------+----+-----------+
|1         |34  |Cardiology |
|2         |NULL|Neurology  |
|3         |50  |Orthopedics|
|4         |NULL|NULL       |
|5         |15  |Neurology  |
+----------+----+-----------+



In [19]:
mean_val=ddF.select(mean(ddF.age)).head()[0]
test = ddF.na.fill(mean_val,subset=['age']).fillna(value = "Unknown",subset=['diagnosis'])


In [20]:
test.show()

+----------+---+-----------+
|patient_id|age|  diagnosis|
+----------+---+-----------+
|         1| 34| Cardiology|
|         2| 33|  Neurology|
|         3| 50|Orthopedics|
|         4| 33|    Unknown|
|         5| 15|  Neurology|
+----------+---+-----------+

